In [10]:
#브라우저 실행
from selenium import webdriver
driver = webdriver.Chrome('C:/Users/soldesk/bigdata/chromedriver.exe')

In [11]:
#상장회사 검색
driver.get('http://marketdata.krx.co.kr/mdi#document=040601')

In [12]:
#다운로드 버튼을 클릭
from selenium.webdriver.common.by import By
button = driver.find_element(By.XPATH, '//button[text()="Excel"]')
button.click()

In [16]:
#다운로드 폴더로 이동
import os
import time
folder='C:\\Users\\soldesk\\Downloads'
os.chdir(folder)

In [17]:
#파일 다운로드까지 대기(1초씩 최대 30회)
fname='data.xls'
for _ in range(30):
    if os.path.exists(fname):
        break
    time.sleep(1)

In [ ]:
#파일명 바꾸기
os.rename('data.xls','상장회사목록.xls')

In [27]:
os.chdir('C:/Users/soldesk/bigdata')
os.getcwd()

'C:\\Users\\soldesk\\bigdata'

In [32]:
import pandas as pd
df_삼성전자 = pd.read_excel('2017년 광고 삼성전자.xlsx')
df_삼성전자.set_index('date', inplace=True)
df_LG전자 = pd.read_excel('2017년 광고 엘지전자.xlsx')
df_LG전자.set_index('date', inplace=True)

In [33]:
#이름만 merge이지, merge가 아니라 빈 dataframe에 insert 한 것.
df_merge = pd.DataFrame()
df_merge['삼성전자'] = df_삼성전자['total']
df_merge['LG전자'] = df_LG전자['total']

In [34]:
#xlsx로 export.
df_merge.to_excel('merged_01.xlsx')

In [38]:
import openpyxl

#엑셀 파일 열고, 시트 얻기
wb = openpyxl.load_workbook('merged_01.xlsx')
sheet = wb.active
sheet

<Worksheet "lemon">

In [39]:
#삼성전자 컬럼
삼성전자_월광고비 = [row[0].value for row in sheet['B2':'B13']]
삼성전자_월광고비

[2845600,
 2763100,
 2679000,
 2763600,
 2731300,
 2825100,
 2863200,
 2846000,
 2817200,
 2838500,
 2852800,
 2888400]

In [41]:
sum(삼성전자_월광고비)

33713800

In [42]:
#합계
sheet['A14'].value='합계'

In [43]:
#합계 계산
sheet['B14'].value= sum([row[0].value for row in sheet['B2':'B13']])
sheet['C14'].value= sum([row[0].value for row in sheet['C2':'C13']])

In [45]:
#가공한 후의 위의 내용이 전부 저장되는 것.
wb.save("merged_02.xlsx")

In [46]:
sheet['B14'].value= '=SUM(B2:B13)'
sheet['C14'].value= '=SUM(C2:C13)'

In [47]:
wb.save("merged_03.xlsx")

In [83]:
from openpyxl.styles import Font, Alignment, Border, Side, Color, PatternFill
from openpyxl.cell import cell
#font 특성 지정
font_15=Font(name='맑은 고딕', size=15, bold=True)
#Alignment: 정렬
align_center = Alignment(horizontal='center', vertical='center')
align_vcenter = Alignment(vertical='center')

In [61]:
#border=테두리 지정
border_thin =Border(
    left=Side(style='thin'), right=Side(style='thin'),
    top=Side(style='thin'), bottom=Side(style='thin')
)

#patternfill=셀 색상 지정
fill_orange=PatternFill(patternType='solid', fgColor=Color('FFC000'))
fill_lightgrey=PatternFill(patternType='solid', fgColor=Color('D3D3D3'))

In [62]:
cell_sum = sheet['A14'] #합계 제목 셀
cell_sum.font=font_15
cell_sum.alignment=align_center
cell_sum.border=border_thin
cell_sum.fill=fill_orange
#type(cell_sum)

In [63]:
wb.save("merged_04.xlsx")

In [71]:
for row in sheet['B2:C14']:
    for cell in row:
        cell.border=border_thin
        cell.number_format='0.00'
for row in sheet['B14:C14']:
    for cell in row:
        cell.alignment=align_vcenter
        cell.fill=fill_orange

In [86]:
column_widths = []

for row in sheet['B2:C14']:
    for i, cell in enumerate(row):
        if len(column_widths) > i:
            if len(cell) > column_widths[i]:
                column_widths[i] = len(cell)
        else:
            column_widths += [len(cell)]

for i, column_width in enumerate(column_widths):
    sheet.column_dimensions[get_column_letter(i+1)].width = column_width

TypeError: object of type 'Cell' has no len()

In [108]:
for col in sheet['A2:C28']: #col의 type=tuple (shift+tab을 눌러 자료형 열람가능)
    max_length = 0
    for cell in col:
        try: # Necessary to avoid error on empty cells
             if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
             pass
    adjusted_width = (max_length + 2) * 15
    for i in range(3):
        sheet.column_dimensions[col[i].column].width = adjusted_width #cell 때문에 column을 가져와야 한다.

In [109]:
wb.save("merged_05.xlsx")

In [110]:
#위의 코드와 합쳐도 무방하지만, 여기에서는 혼동을 막기 위해 분리해보았다.
for col in sheet['A2:C28']:
    for cell in range(3):
        if(cell%2==0):
            col[cell].font=font_15
        else:
            pass

In [111]:
wb.save("merged_05.xlsx")